In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
tqdm.pandas()

In [ ]:
yc_reviews = pd.read_pickle("../data/pickles/yelpchi_reviews.pkl")
yc_updated_reviews = pd.read_pickle("../data/pickles/yelpchi_updated_reviews.pkl")

In [ ]:
len(yc_reviews)+len(yc_updated_reviews)

In [ ]:
len(yc_reviews[yc_reviews.matchID.isnull()])+len(yc_updated_reviews)

In [ ]:
(len(yc_updated_reviews[yc_updated_reviews.flagged == "N"]) + len(yc_reviews[yc_reviews.flagged == "N"])) / (len(yc_reviews) + len(yc_updated_reviews))

In [ ]:
(len(yc_reviews[yc_reviews.flagged == "N"])) / (len(yc_reviews)),(len(yc_updated_reviews[yc_updated_reviews.flagged == "N"])) / (len(yc_updated_reviews))

In [ ]:
yc_reviews.date.max(),yc_updated_reviews.date.max()

In [ ]:
list(yc_updated_reviews),list(yc_reviews)

In [ ]:
#Let's unify author IDs

In [ ]:
old_to_new_ids = {}
for _, row in tqdm(yc_reviews[yc_reviews.matchID.notnull()].iterrows(),total=len(yc_reviews)):
    if row.reviewerID not in old_to_new_ids:
        old_to_new_ids[row.reviewerID] = {}
    new_match = yc_updated_reviews.loc[row.matchID]
    if pd.notnull(new_match.user_page_url):
        old_to_new_ids[row.reviewerID]["r"] = new_match.user_page_url
    else:
        old_to_new_ids[row.reviewerID]["n"] = new_match.data_hovercard_id
        

In [ ]:
author_id_mapping = {}

for _, mapping in old_to_new_ids.items():
    if "r" in mapping and "n" in mapping:
        author_id_mapping[mapping["n"]] = mapping["r"]

In [ ]:
def get_author_id(row):
    if pd.notnull(row.user_page_url):
        return row.user_page_url
    if row.data_hovercard_id in author_id_mapping:
        return author_id_mapping[row.data_hovercard_id]
    return row.data_hovercard_id
    
yc_updated_reviews["author_id"] = yc_updated_reviews.progress_apply(get_author_id,axis=1)

In [ ]:
len(yc_updated_reviews["author_id"].unique()),len(yc_updated_reviews[~yc_updated_reviews["author_id"].isin(yc_updated_reviews.data_hovercard_id.unique())].author_id.unique())

In [ ]:
len(yc_updated_reviews.data_hovercard_id.unique()),len(yc_updated_reviews.user_page_url.unique())